In [1]:
from owlready2 import *
world = World()
#world.set_backend(filename = "./world_extract.sqlite3", exclusive = False) ##use concurrently

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
import pandas as pd

In [3]:
tao_base_iri = "http://purl.org/tao/ns#"
tao_ontology = world.get_ontology("output_ontology/tao.rdf").load()
skos_file = './ontologies/skos.rdf' 
skos_ontology = world.get_ontology(skos_file).load()
skos = skos_ontology.get_namespace("http://www.w3.org/2004/02/skos/core#")
tao_solo = tao_ontology.get_namespace(tao_base_iri)

In [4]:
tao_solo.HypoallergenicRoom.label

['Hypoallergenic room']

In [5]:
#graph = world.as_rdflib_graph()
# r = list(graph.query("""
# SELECT ?s ?l WHERE {
#     ?b  <http://www.w3.org/2002/07/owl#annotatedSource> ?s ;
#         <http://www.w3.org/2002/07/owl#annotatedTarget> ?l ;
#         <http://www.w3.org/2002/07/owl#annotatedProperty> <http://www.w3.org/2004/02/skos/core#altLabel> .
# }
# """))

In [6]:
world._get_by_storid(337).iri

'http://purl.org/acco/ns#HotelRoom'

In [7]:
tao_solo.Room

tao.Room

In [8]:

for c in tao_solo.classes():
    parents = ""
    for parent in c.is_a:
        if type(parent) is owlready2.entity.ThingClass:
            parents = parents + parent.name +"|"
    parents = parents[:-1]
    if c.iri.startswith(tao_base_iri):
        print(c.name+","+parents+","+ str(c.storid))

LodgingFacility,Location|Feature|Place|Thing,316
NormAggregateRating,AggregateRating,317
Accommodation,Accommodation,323
BathroomAmenity,AccommodationFeature|LocationAmenity,346
LocationAmenity,Thing,347
CampingPitch,CampingPitch|Accommodation,350
Campground,LodgingFacility,351
EntireApartment,Apartment|EntirePlace,352
EntireHouse,House|EntirePlace,353
Room,Accommodation,354
Suite,Accommodation,355
EntirePlace,Accommodation,358
Dryer,BathroomAmenity|AccommodationFeature,363
EntireBoat,EntirePlace,366
EntireLoft,EntirePlace,368
HighNormRating,NormAggregateRating,370
HighRatedLF,RatedLF,371
RatedLF,LodgingFacility,372
Hotel,LodgingFacility,373
House,LodgingFacility,374
HotelRoom,Room,380
LodgingDescription,CreativeWork,383
LowNormRating,NormAggregateRating,385
LowRatedLF,RatedLF,386
MediumNormRating,NormAggregateRating,387
MediumRatedLF,RatedLF,388
NormRating,Rating,389
TouristLocation,Place|Feature,391
TouristDestination,Feature,393
LocalBusiness,TouristLocation,394
FoodEstablishment,Lo

In [9]:
Accommodation = tao_solo.Accommodation

In [10]:
Accommodation.subclasses()

<generator object EntityClass.subclasses at 0x7f28b14c2890>

In [11]:
class_iri = tao_solo.Barbecue.iri
res = list(world.sparql("""
    SELECT ?l WHERE {
    ?b  owl:annotatedSource <%s> ;
        owl:annotatedTarget ?l ;
        owl:annotatedProperty skos:altLabel .
        }
    """ % class_iri))

In [12]:
res1 = [l[0] for l in res]

In [13]:
res1

['BBQ facilities', 'BBQ grill']

In [14]:
def get_all_labels(owl_class, languages = ["en"]):
    ##TODO: generalize to extract labels from a list of annotation properties passed as an argument
    candidate_labels = []
    labels = []
    rdfs_labels = list(owl_class.label)
    try:
        skos_altLabels = list(owl_class.altLabel) 
    except AttributeError:
        print("######## skos!!!")
        skos_altLabels = []
    skos_annotated_labels = get_annotated_labels(owl_class, skos.altLabel)
    candidate_labels = rdfs_labels + skos_altLabels + skos_annotated_labels
    #print("examining labels")
    #print(candidate_labels)
    for label in candidate_labels:
        try:
            lang = label.lang
            #print("lang:", lang)
            if lang not in languages:
                #print("###########")
                continue
        except Exception as e:
            print(e)
            #pass
        if label.lower() not in labels: ## avboid duplicated labels
            labels.append(label.lower())
        #else:
            #print("already known", label)
            
    return labels

def get_annotated_labels(owl_class, owl_property = skos.altLabel):
    class_iri = owl_class.iri
    prop = owl_property.iri
    res = list(world.sparql("""
        SELECT ?l WHERE {
        ?b  owl:annotatedSource <%s> ;
            owl:annotatedTarget ?l ;
            owl:annotatedProperty <%s> .
            }
        """ % (class_iri, prop)))
    flat_res = [l[0] for l in res]
    return flat_res

def print_level(base_class, tab = "-", languages = ["en"]):
    for c in base_class.subclasses():
        print(tab, c.name)
        labels = get_all_labels(c)
        print("labels:", labels)
        if len(list(c.subclasses())) > 0:
            print_level(c, "-"+tab)

def yield_labels(owl_class, languages = ["en"]):
    labels = get_all_labels(owl_class)
    for label in labels:
        yield label, owl_class.name

def _extract_mapping_table(base_class, languages = ["en"]):
    #print("examining class", flush = True)
    #print("examining class", base_class.name, flush=True)
    class_and_subclasses = [base_class]
    for c in base_class.subclasses():
        #print("subclass: ", c.name)
        yield from yield_labels(c, languages = languages)
        #labels = get_all_labels(c)
        #for label in labels:
        #    yield label, c.name
        #print("Looking sublcasses of", c.name)    
        yield from _extract_mapping_table(c, languages = languages)
        
def extract_mapping_table(base_class, languages = ["en"]):
    yield from yield_labels(base_class, languages = languages) ## extract labels from the top class  
    yield from _extract_mapping_table(base_class, languages = languages) ## extract labels from the subclasses recursively

In [15]:
get_annotated_labels(tao_solo.Barbecue, skos.altLabel)

['BBQ facilities', 'BBQ grill']

In [16]:
#print_level(tao_solo.Hotel)

In [17]:
#print_level(tao_solo.LodgingFacility)

In [18]:
print_level(tao_solo.LocationAmenity)

- BathroomAmenity
labels: ['bathroom amenity']
-- Dryer
labels: ['dryer', 'dryer – in building', 'free dryer', 'free dryer – in unit', 'tumble dryer']
-- ShowerBath
labels: ['shower', 'shower bath']
-- BathRobes
labels: ['bath robes']
-- HygieneProducts
labels: ['hygiene products']
-- Towel
labels: ['towel', 'essentials - towels']
-- Bathtub
labels: ['bathtub', 'hot tub']
-- Toilet
labels: ['toilet']
--- ToiletWithGrabRails
labels: ['toilet with grab rails']
--- RaisedToilet
labels: ['raised toilet']
--- HigherLevelToilet
labels: ['higher level toilet']
-- Balance
labels: ['balance']
-- Tap
labels: ['faucet', 'tap']
-- Sink
labels: ['washbasin', 'sink']
--- LoweredSink
labels: ['lowered sink']
-- Bidet
labels: ['bidet']
-- Hairdryer
labels: ['hairdryer']
-- BathroomEmergencyCord
labels: ['bathroom emergency cord', 'emergency cord in bathroom']
-- ToiletPaper
labels: ['toilet paper']
-- SharedBathroom
labels: ['shared bathroom']
-- FreeToiletries
labels: ['free toiletries']
-- SpaBath
l

In [19]:
df_amenity_map = pd.DataFrame(list(extract_mapping_table(tao_solo.LocationAmenity)) ,
               columns =['label', 'class'])
df_amenity_map.drop_duplicates(subset='label', keep="last", inplace=True)

In [20]:
df_amenity_map.shape

(512, 2)

In [21]:
df_amenity_map.to_csv("class_mapping/la_ontology_mapping.csv", index=False)

In [22]:
df_acco_map = pd.DataFrame(list(extract_mapping_table(tao_solo.Accommodation)) ,
               columns =['label', 'class'])
df_acco_map.drop_duplicates(subset='label', keep="last", inplace=True)

In [23]:
df_acco_map

,label,class
0,accommodation,Accommodation
1,camping pitch,CampingPitch
2,room,Room
3,hotel room,HotelRoom
4,shared room,SharedRoom
5,meeting room,MeetingRoom
6,bedroom,Bedroom
7,living room,LivingRoom
8,private living room,LivingRoom
9,dining room,DiningRoom


In [24]:
df_acco_map.to_csv("class_mapping/accommodation_ontology_mapping.csv", index=False)

In [25]:
df_lf_map = pd.DataFrame(list(extract_mapping_table(tao_solo.LodgingFacility)),
               columns =['label', 'class'])
df_lf_map.drop_duplicates(subset='label', keep="last", inplace=True)

In [26]:
df_lf_map.to_csv("class_mapping/lf_ontology_mapping.csv", index=False)

In [27]:
df_lf_map

,label,class
0,lodging facility,LodgingFacility
1,campground,Campground
2,campsite,Campground
3,hotel,Hotel
4,boutique hotel,Hotel
10,contry house hotel,CountryHouseHotel
11,contry house,CountryHouseHotel
12,cave hotel,CaveHotel
13,capsule hotel,CapsuleHotel
14,under water hotel,UnderWaterHotel
